Lab 2

Q1

In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from scipy.stats import zscore

In [78]:
data = pd.read_csv("../../practical_labs/Lab2_dataset.csv")
data.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\n( see a...",0
2,3624,ham,"Subject: neon retreat\nho ho ho , we ' re arou...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\nthis deal is to ...,0


In [79]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(
    data["text"]
).toarray()  
y = data["label_num"]

In [80]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=31
)

In [81]:
gmodel = GaussianNB()
gmodel.fit(X_train, y_train)

gpred = gmodel.predict(X_test)
print(gpred)
y_test

[0 0 1 ... 0 0 1]


987     0
547     0
2570    1
1587    1
134     1
       ..
347     0
2615    0
658     0
1977    0
855     1
Name: label_num, Length: 1293, dtype: int64

In [82]:
mmodel = MultinomialNB()
mmodel.fit(X_train, y_train)

mpred = mmodel.predict(X_test)
print(mpred)
y_test

[0 0 1 ... 0 0 1]


987     0
547     0
2570    1
1587    1
134     1
       ..
347     0
2615    0
658     0
1977    0
855     1
Name: label_num, Length: 1293, dtype: int64

In [83]:
print("Gaussian Accuracy :")
print(accuracy_score(y_test,gpred,))

Gaussian Accuracy :
0.954369682907966


In [84]:
print("Multinomial Accuracy :")
print(accuracy_score(y_test,mpred,))

Multinomial Accuracy :
0.8808971384377416


For the Gaussian classifier the accuracy is 95.43% and from Multinomial the accuracy is 88.08%. The predictions are verified by comparing the test value against predicted value.

Q2

In [85]:
data = pd.read_csv("../../practical_labs/AB_NYC_2019.csv")
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Removing NA values

In [86]:
cols = ["price", "minimum_nights", "number_of_reviews", "availability_365"]

data[cols] = data[cols].dropna()

Using percentile

In [87]:
Q1 = data[cols].quantile(0.25)
Q3 = data[cols].quantile(0.75)
IQR = Q3 - Q1

outlier percentile

In [88]:
lw = Q1 - 1.5 * IQR
print(lw)
uw = Q3 + 1.5 * IQR
print(uw)
outlier_rows_percentile = data[
    (
        (data[cols] < lw) | (data[cols] > uw)
    ).any(axis=1)
].index
outlier_rows_percentile

price                -90.0
minimum_nights        -5.0
number_of_reviews    -33.5
availability_365    -340.5
dtype: float64
price                334.0
minimum_nights        11.0
number_of_reviews     58.5
availability_365     567.5
dtype: float64


Index([    3,     5,     6,     7,     8,     9,    11,    12,    13,    14,
       ...
       48808, 48810, 48833, 48839, 48842, 48843, 48856, 48871, 48879, 48882],
      dtype='int64', length=14820)

Using z-score

In [89]:
z_scores = np.abs(zscore(data[cols]))
z_scores

,price,minimum_nights,number_of_reviews,availability_365
0,0.015493,0.293996,0.320414,1.916250
1,0.300974,0.293996,0.487665,1.840275
2,0.011329,0.196484,0.522433,1.916250
3,0.265335,0.293996,5.538156,0.617065
4,0.302811,0.144807,0.320414,0.856865
...,...,...,...,...
48890,0.344452,0.245240,0.522433,0.788486
48891,0.469373,0.147729,0.522433,0.583352
48892,0.157070,0.144807,0.522433,0.651730
48893,0.406912,0.293996,0.522433,0.841669


z-scores

In [90]:
outlier_zscore = np.where(z_scores > 3)[0]
outlier_zscore

array([    3,     7,     9, ..., 48446, 48523, 48535])

Removing outliers

In [91]:
bclean = data.drop(outlier_zscore)
bclean.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
5,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


Another method is to replace outliers with mean, median or mode values.
Using mean to replace outliers.

In [92]:
indicator = z_scores > 3

bdclean = data
bdclean[cols] = np.where(
    indicator, data[cols].mean(), data[cols]
)

bdclean.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149.0,1.0,9.000000,2018-10-19,0.21,6,365.0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225.0,1.0,45.000000,2019-05-21,0.38,2,355.0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150.0,3.0,0.000000,NaN,NaN,1,365.0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89.0,1.0,23.274466,2019-07-05,4.64,1,194.0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80.0,10.0,9.000000,2018-11-19,0.10,1,0.0
